In [1]:
import nltk.stem
import re
import numpy as np
import os
import nltk
import pandas as pd
from src.SemEvalData import SemEvalData
from src.JigsawData import JigsawData
from src.preprocessing import preprocess_bayes
import joblib
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#stemmer = nltk.stem.SnowballStemmer('english')
lemmer = nltk.stem.WordNetLemmatizer()

In [4]:
## load data
train_data_semeval = SemEvalData()
train_data_semeval.load_data("data/tsd_trial.csv")
train_df_preprocessed = train_data_semeval.preprocess()

                                               spans  \
0           [15, 16, 17, 18, 19, 27, 28, 29, 30, 31]   
1                           [29, 30, 31, 32, 33, 34]   
2                     [166, 167, 168, 169, 170, 171]   
3                           [87, 88, 89, 90, 91, 92]   
4                                                 []   
5                                 [8, 9, 10, 11, 12]   
6  [265, 266, 267, 268, 269, 270, 271, 272, 273, ...   
7                                                 []   
8                           [38, 39, 40, 41, 42, 43]   
9  [277, 278, 279, 280, 281, 282, 283, 284, 285, ...   

                                                text  
0  Because he's a moron and a bigot. It's not any...  
1  How about we stop protecting idiots and let na...  
2  If people  were  smart, they would  Boycott th...  
3  Trump Claimed that Russia will never invade th...  
4  As long as your willing to pay a lot more for ...  
5  Only an idiot would use and believe anything t... 

Name: 32, dtype: object
[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135]
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 46
46 47
47 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 56
56 57
57 58
58 59
59 92
92 93
93 94
94 95
95 96
96 97
97 98
98 99
99 100
100 101
101 102
102 103
103 104
104 105
105 106
106 107
107 108
108 109
109 110
110 111
111 112
112 113
113 114
114 115
115 116
116 117
117 118
118 119
119 120
120 121
121 122
122 123
123 124
124 125
125 126
126 127
127 128
128 129
129 130
130 131
131 132
132 133
133 134
134 135
2 [[35, 59], [92, 135]]
spans                                                      []
text        These freaking donkeys all need to be removed ...
toxicity                            

1 [[72, 77]]
spans            [11, 12, 13, 14, 15, 16]
text        Muslims would beg to disagree
toxicity                                1
Name: 83, dtype: object
[11, 12, 13, 14, 15, 16]
11 12
12 13
13 14
14 15
15 16
1 [[11, 16]]
spans                     [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
text        This ridiculous nonsense needs to stop.  It's ...
toxicity                                                    1
Name: 84, dtype: object
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
1 [[5, 14]]
spans       [0, 1, 2, 3, 4, 5, 6, 7]
text                        You suck
toxicity                           1
Name: 85, dtype: object
[0, 1, 2, 3, 4, 5, 6, 7]
0 1
1 2
2 3
3 4
4 5
5 6
6 7
1 [[0, 7]]
spans       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 176, ...
text        Survival of the fittest would not have produce...
toxicity                                                    1
Name: 86, dtype: object
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 176, 177, 178, 1

spans       [8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 23, 24,...
text        This is either dumb or stupid.\n\nThe issue mo...
toxicity                                                    1
Name: 140, dtype: object
[8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 23, 24, 25, 26, 27, 28]
8 9
9 10
10 11
11 12
12 13
13 15
15 16
16 17
17 18
18 23
23 24
24 25
25 26
26 27
27 28
3 [[8, 13], [15, 18], [23, 28]]
spans       [380, 381, 382, 383, 384, 385, 386, 387, 388, ...
text        There is no bill. The 2% figure is a guideline...
toxicity                                                    1
Name: 141, dtype: object
[380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446]
380 381
381 382
382 383
383 384
384 385
385 386
386 387
387 

15 16
16 17
17 18
1 [[14, 18]]
spans       [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 4...
text        Are these some of those Russian pieces of crap...
toxicity                                                    1
Name: 183, dtype: object
[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 116, 117, 118, 119, 120, 121]
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 116
116 117
117 118
118 119
119 120
120 121
2 [[32, 45], [116, 121]]
spans                                    [66, 67, 68, 69, 70]
text        Nurnie/Saullie: and you're back with your fool...
toxicity                                                    1
Name: 184, dtype: object
[66, 67, 68, 69, 70]
66 67
67 68
68 69
69 70
1 [[66, 70]]
spans                                                      []
text        Like I said you can't grasp it. We have to put...
toxicity                                                    0
Name: 185, dtype: object
spans                               [273,

...

45 46
46 47
47 48
48 49
49 50
50 51
51 52
1 [[44, 52]]
spans                                         [4, 5, 6, 7, 8]
text        The idiot should be fired..... I personally wo...
toxicity                                                    1
Name: 502, dtype: object
[4, 5, 6, 7, 8]
4 5
5 6
6 7
7 8
1 [[4, 8]]
spans                                    [22, 23, 24, 25, 26]
text        Trudeau is a complete idiot just like his old ...
toxicity                                                    1
Name: 503, dtype: object
[22, 23, 24, 25, 26]
22 23
23 24
24 25
25 26
1 [[22, 26]]
spans                          [102, 103, 104, 105, 106, 107]
text        Bunch of entitled whiners and beggars...but on...
toxicity                                                    1
Name: 504, dtype: object
[102, 103, 104, 105, 106, 107]
102 103
103 104
104 105
105 106
106 107
1 [[102, 107]]
spans                                    [135, 136, 137, 138]
text        Pence managed the transition.\nThere is no way...
t

2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
1 [[0, 19]]
spans                                    [21, 22, 23]
text        Please don't go away mad....just go away!
toxicity                                            1
Name: 554, dtype: object
[21, 22, 23]
21 22
22 23
1 [[21, 23]]
spans               [13, 14, 15, 16, 17, 18, 19, 20, 21]
text        You win, The stupidest comment of the year .
toxicity                                               1
Name: 555, dtype: object
[13, 14, 15, 16, 17, 18, 19, 20, 21]
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
1 [[13, 21]]
spans       [13, 14, 15, 16, 17, 18, 191, 192, 193, 194, 1...
text        Yeah, pretty stupid black&white picture of the...
toxicity                                                    1
Name: 556, dtype: object
[13, 14, 15, 16, 17, 18, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215]
13 14
14 15
15 16
16 

Name: 601, dtype: object
[35, 36, 37, 38, 39, 40]
35 36
36 37
37 38
38 39
39 40
1 [[35, 40]]
spans       [67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7...
text        Good morning, ducati, Moto guzzi, triumph and ...
toxicity                                                    1
Name: 602, dtype: object
[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86]
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
1 [[67, 86]]
spans       [8, 9, 10, 11, 12, 13]
text        another stupid comment
toxicity                         1
Name: 603, dtype: object
[8, 9, 10, 11, 12, 13]
8 9
9 10
10 11
11 12
12 13
1 [[8, 13]]
spans              [3, 4, 5, 6, 7]
text        An idiot from new york
toxicity                         1
Name: 604, dtype: object
[3, 4, 5, 6, 7]
3 4
4 5
5 6
6 7
1 [[3, 7]]
spans                                         [340, 341, 342]
text        Lol, such an idiotic comment.  There's junk fo...

Name: 635, dtype: object
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
1 [[0, 38]]
spans       [64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 7...
text        I'd happily take Nixon, and both Bushies at th...
toxicity                                                    1
Name: 636, dtype: object
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 93
93 94
94 95
95 96
96 97
97 98
98 99
99 100
100 101
101 102
1 [

49 50
50 51
51 52
52 53
1 [[44, 53]]
spans       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
text                         You're a fake, alright...
toxicity                                             1
Name: 678, dtype: object
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
1 [[0, 12]]
spans                     [124, 125, 126, 127, 128, 129, 130]
text        Grand standing egomaniacs?   Oh I get it, unli...
toxicity                                                    1
Name: 679, dtype: object
[124, 125, 126, 127, 128, 129, 130]
124 125
125 126
126 127
127 128
128 129
129 130
1 [[124, 130]]
spans       [153, 154, 155, 156, 157, 158, 159, 160, 161, ...
text        Are you a Woman? No? Then why are you commenti...
toxicity                                                    1
Name: 680, dtype: object
[153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165]
153 154
154 155
155 156
156 157
157 158
158 159
159 160
160 161
161 162
162 16

In [5]:
train_data = {
    'sentence':  train_df_preprocessed.sentences.sum(),
    'toxicity_sentence': train_df_preprocessed.toxicity_sentence.sum()
        }

train_df = pd.DataFrame (train_data, columns = ['sentence','toxicity_sentence'])

In [6]:

train_df['sentence'] = train_df.apply(lambda row: preprocess_bayes(row.sentence), axis=1)
print(train_df.head(5))

                                            sentence  toxicity_sentence
0  because hes a moron and a bigot its not any mo...                1.0
1  how about we stop protecting idiots and let na...                1.0
2  if people  were  smart they would  boycott thi...                1.0
3  trump claimed that russia will never invade th...                1.0
4  as long as your willing to pay a lot more for ...                0.0


In [7]:
def stemming(text):
    # Stem words
    #data[i] = ' '.join([stemmer.stem(word) for word in data[i].split()])
    data = ' '.join([lemmer.lemmatize(word) for word in text.split()])
    return data


In [29]:
# Perform a grid search to find the best hyperparameters
def grid_search(train):
    # Create a pipeline
    clf_pipeline = sklearn.pipeline.Pipeline([
        ('v', sklearn.feature_extraction.text.CountVectorizer(strip_accents='ascii', stop_words='english')),
        ('t', sklearn.feature_extraction.text.TfidfTransformer()), 
        ('c', sklearn.naive_bayes.MultinomialNB(fit_prior=True, class_prior=None))
        ])
    # Set parameters (name in pipeline + name of parameter)
    parameters = { 
        'v__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)], 
        'v__lowercase': (True, False), 
        't__use_idf': (True, False), 
        'c__alpha': (0.3, 0.6, 1.0) }
    # Create a grid search classifier
    gs_classifier = sklearn.model_selection.GridSearchCV(clf_pipeline, parameters, cv=5, iid=False, n_jobs=2, scoring='accuracy', verbose=1)
    
    # Start a search (Warning: takes a long time if the whole dataset is used)
    # Slice: (train.data[:4000], train.target[:4000])
    gs_classifier = gs_classifier.fit(train.data, train.target)
    # Print results
    print('---- Results ----')
    print('Best score: ' + str(gs_classifier.best_score_))
    for name in sorted(parameters.keys()):
        print('{0}: {1}'.format(name, gs_classifier.best_params_[name]))

In [8]:
train_df['sentence'] = train_df.apply(lambda row: stemming(row.sentence), axis=1)

In [9]:
print(train_df.head())

                                            sentence  toxicity_sentence
0  because he a moron and a bigot it not any more...                1.0
1  how about we stop protecting idiot and let nat...                1.0
2  if people were smart they would boycott this i...                1.0
3  trump claimed that russia will never invade th...                1.0
4  a long a your willing to pay a lot more for pr...                0.0


In [63]:
# Train and evaluate a model
def train_and_evaluate(train):
    
    # Convert to bag of words
    count_vect = sklearn.feature_extraction.text.CountVectorizer(strip_accents='ascii', stop_words='english', lowercase=True, ngram_range=(1,1))
    X = count_vect.fit_transform(train['sentence'])
    # Convert from occurrences to frequencies
    # Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
    # To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.
    transformer = sklearn.feature_extraction.text.TfidfTransformer()
    X = transformer.fit_transform(X)
    # Create a model
    # get the first vector out (for the first document) 
    first_vector_tfidfvectorizer=X[0] 
 
    # place tf-idf values in a pandas data frame 
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=count_vect.get_feature_names(), columns=["tfidf"]) 
    df.sort_values(by=["tfidf"],ascending=True)
    print(df.head)
    model = sklearn.naive_bayes.MultinomialNB(alpha=0.3, fit_prior=True, class_prior=None)
    # Train the model
    model.fit(X, train['toxicity_sentence'])
    # Save models
    joblib.dump(count_vect, 'vectorizer.jbl')
    joblib.dump(transformer, 'transformer.jbl')
    joblib.dump(model, 'model.jbl')
    # Evaluate on training data
    print('-- Training data --')
    predictions = model.predict(X)
    print(model.coef_, len(count_vect.get_feature_names()))
#     neg_class_prob_sorted = model.feature_log_prob_[0, :].argsort()
#     pos_class_prob_sorted = model.feature_log_prob_[1, :].argsort()

#     print(np.take(count_vect.get_feature_names(), neg_class_prob_sorted[:10]))
#     print(np.take(count_vect.get_feature_names(), pos_class_prob_sorted[:10]))
#     print(predictions,X, train['sentence'])
#     accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
#     print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
#     print('Classification Report:')
#     print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))
#     print('')
    # Evaluate with 10-fold CV
#     print('-- 10-fold CV --')
#     predictions = sklearn.model_selection.cross_val_predict(model, X, train['toxicity_sentence'], cv=10)
#     accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
#     print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
#     print('Classification Report:')
#     print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))


In [64]:
train_and_evaluate(train_df)

<bound method NDFrame.head of           tfidf
aa          0.0
ability     0.0
abject      0.0
able        0.0
abortion    0.0
...         ...
zion        0.0
zip         0.0
zombie      0.0
zone        0.0
zygote      0.0

[4125 rows x 1 columns]>
-- Training data --
[[-8.79875796 -8.63101496 -8.49786284 ... -8.77176987 -8.93436028
  -9.05984196]] 4125
